In [9]:
import tweepy
import pandas as pd
import json
import datetime as dt
import os
import sys
import time

In [10]:
consumer_key = 'KwsM7g4X8QPXdGtAXpGf9zIVD'
consumer_secret = 'mW6Ru1IxyyXGxIaFKL7YfWJC1dZWbHYWTA7bIdpMGqVL4hIBZd'
access_token = '1055276480635265024-3HGLjEGA9wewgSuT9Pskio0Rhmr1H2'
access_token_secret = '0e2lJBV2EHnMeOGm0QjE7X6SrX87ouDozANbdw8lDsRGo'

In [11]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
## heavily borrowed from 
# https://github.com/agalea91/twitter_search/blob/master/twitter_search.py

In [12]:
time_limit = 1.5                           # runtime limit in hours
max_tweets = 100                           # number of tweets per search (will be
                                           # iterated over) - maximum is 100
max_days_old = 8                           # search limits e.g., from 7 to 8
                                           # gives current weekday from last week,
                                           # min_days_old=0 will search from right now
USA = '39.8,-95.583068847656,2500km'       # this geocode includes nearly all American
                                           # states (and a large portion of Canada)
query_list = ["juul","#Juul","juuling","#switchtojuul"]
#query_list = ['juuling','juul','#e-cigarettes', '#vape'] #'#switchtojuul','#juul', 
#read_IDs = False # initialize - first round of tweets added to json

In [13]:
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q = query, count = remaining_tweets,\
                               since_id = str(since_id), max_id = str(max_id-1),\
                                   geocode = geocode)
            if not new_tweets: # does new_tweets have a value
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets) # adds new tweets
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('waiting 15 minutes due to time limit')
            print(dt.datetime.now()+dt.timedelta(minutes=15))
            time.sleep(15*60)
            break       
    return searched_tweets, max_id

In [14]:
def scrape_tweets(query, day, max_id, since_id, time_limit, max_tweets, USA):
    json_file = query.replace('#','hshtg_') + '_' + day + '.json'
    print(json_file) 

    api = tweepy.API(auth)

    start = dt.datetime.now()
    end = start + dt.timedelta(hours=time_limit)
    count, exitcount = 0, 0
    while dt.datetime.now() < end:
        count += 1
        print('count =',count)
        # collect tweets and update max_id
        tweets, max_id = tweet_search(api=api, query=query, max_tweets=max_tweets,
                                      max_id=max_id, since_id=since_id,
                                      geocode=USA)
        # write tweets to file in JSON format
        if tweets:
            with open(json_file, 'a') as f:
                for tweet in tweets:
                    json.dump(tweet._json, f)
                    f.write('\n')
            exitcount = 0
        else:
            exitcount += 1
            if exitcount == 3:
                print('Maximum number of empty tweet strings reached - breaking')
                break
    return json_file

In [15]:
for q in query_list:
    api = tweepy.API(auth)
    
    d1 = dt.datetime.now() - dt.timedelta(days=max_days_old)
    print('start date:','\n',d1)
    tweet_date_1 = '{0}-{1:0>2}-{2:0>2}'.format(d1.year, d1.month, d1.day)
    # get list of up to 10 tweets
    tweet_1 = api.search(q=q, count=1, until=tweet_date_1, geocode=USA)
    since_id = tweet_1[0].id
    print('start ID:','\n',since_id)

    d2 = dt.datetime.now() # if someone didnt tweet
    print('end date:','\n',d2)
    tweet_date_2 = '{0}-{1:0>2}-{2:0>2}'.format(d2.year, d2.month, d2.day)
    # get list of up to 10 tweets
    tweet_2 = api.search(q=q, count=10, until=tweet_date_2, geocode=USA)
    max_id = tweet_2[0].id
    print('end ID:','\n',max_id)

    day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
           d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
    print(day)
    
    json_file = scrape_tweets(query=q, day=day,max_id=max_id, since_id=since_id,time_limit=1.5, max_tweets=max_tweets, USA=USA)
    print('finished with query:',q)
    print('file:',json_file)

start date: 
 2018-11-18 18:12:02.727969
start ID: 
 1063944536165101568
end date: 
 2018-11-26 18:12:04.384909
end ID: 
 1066843795352940544
2018-11-18_to_2018-11-26
juul_2018-11-18_to_2018-11-26.json
count = 1
count = 2
count = 3
count = 4
count = 5
count = 6
count = 7
count = 8
count = 9
count = 10
count = 11
count = 12
count = 13
count = 14
count = 15
count = 16
count = 17
count = 18
count = 19
count = 20
count = 21
count = 22
count = 23
no tweets found
count = 24
count = 25
count = 26
count = 27
count = 28
count = 29
count = 30
count = 31
count = 32
count = 33
count = 34
count = 35
count = 36
count = 37
count = 38
count = 39
count = 40
count = 41
count = 42
count = 43
count = 44
count = 45
count = 46
count = 47
count = 48
count = 49
count = 50
count = 51
count = 52
count = 53
count = 54
count = 55
count = 56
count = 57
count = 58
count = 59
count = 60
count = 61
count = 62
count = 63
count = 64
count = 65
count = 66
count = 67
count = 68
count = 69
count = 70
count = 71
count = 72

IndexError: list index out of range

In [38]:
# json_file = 'juul_2018-10-20_to_2018-10-28.json'
# tweet_data = []
# with open(json_file, 'r') as f:
#     for line in f.readlines():
#         tweet_data.append(json.loads(line))

In [39]:
# df = pd.DataFrame()

# #general info
# df['id'] = list(map(lambda tweet: tweet['id_str'], tweet_data))
# df['tweet_date'] = list(map(lambda tweet: tweet['created_at'], tweet_data))
# df['text'] = list(map(lambda tweet: tweet['text'], tweet_data))
# df['lang'] = list(map(lambda tweet: tweet['lang'], tweet_data))
# # user information
# df['location'] = list(map(lambda tweet: tweet['user']['location'], tweet_data))
# df['protected'] = list(map(lambda tweet: tweet['user']['protected'], tweet_data))
# df['profile_description'] = list(map(lambda tweet: tweet['user']['description'], tweet_data))
# df['followers_count'] = list(map(lambda tweet: tweet['user']['followers_count'], tweet_data))
# df['friends_count'] = list(map(lambda tweet: tweet['user']['friends_count'], tweet_data))
# df['listed_count'] = list(map(lambda tweet: tweet['user']['listed_count'], tweet_data))
# df['favourites_count'] = list(map(lambda tweet: tweet['user']['favourites_count'], tweet_data))
# df['geo_enabled'] = list(map(lambda tweet: tweet['user']['geo_enabled'], tweet_data))
# df['verified'] = list(map(lambda tweet: tweet['user']['verified'], tweet_data))
# df['statuses_count'] = list(map(lambda tweet: tweet['user']['statuses_count'], tweet_data))
# # tweet info
# df['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweet_data))
# df['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweet_data))
# df['favorited'] = list(map(lambda tweet: tweet['favorited'], tweet_data))
# df['retweeted'] = list(map(lambda tweet: tweet['retweeted'], tweet_data))
# # tweet location
# df['longitude'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][0] 
#                                   if tweet['coordinates'] != None else 'NaN', tweet_data))
# df['latitude'] = list(map(lambda tweet: tweet['coordinates']['coordinates'][1] 
#                                   if tweet['coordinates'] != None else 'NaN', tweet_data))


In [41]:
#df.head()

,id,tweet_date,text,lang,location,protected,profile_description,followers_count,friends_count,listed_count,favourites_count,geo_enabled,verified,statuses_count,retweet_count,favorite_count,favorited,retweeted,longitude,latitude
0,1056334344787292160,Sat Oct 27 23:58:28 +0000 2018,RT @Sadieisonfire: do u enjoy the crack of a w...,en,66.6 FM,False,emo • your fav bio exorcist • Joseph 💜💜 • G59 ...,464,421,2,3124,True,False,23049,670,0,False,False,NaN,NaN
1,1056334344128733185,Sat Oct 27 23:58:28 +0000 2018,Can’t rlly think of anything more tragic than ...,en,"New York, NY",False,A place for my thots bcuz my friends are sick ...,5,10,0,118,False,False,314,0,0,False,False,NaN,NaN
2,1056334201572605952,Sat Oct 27 23:57:54 +0000 2018,RT @ItsJohnConner: @nytimes They lost to Juul ...,en,"Calgary, Alberta",False,vaper 4yrs +/non-smoker/just a mom/volunteer 4...,358,287,5,17364,False,False,12763,4,0,False,False,NaN,NaN
3,1056334169419251712,Sat Oct 27 23:57:46 +0000 2018,RT @Sadieisonfire: do u enjoy the crack of a w...,en,"st. augustine, fl",False,beyond happy,458,441,2,13460,False,False,3776,670,0,False,False,NaN,NaN
4,1056334111772684290,Sat Oct 27 23:57:32 +0000 2018,Rip to my 9th juul onto my 10th #ripbankaccoun...,en,"Chicago, IL",False,straight chillin,328,189,3,9240,True,False,8353,0,3,False,False,NaN,NaN
